In [1]:
import utils_nlp
import pickle
import ner_model
import tensorflow as tf
import dataset
from dataset import Dataset

C:\Users\linhpn.VISC\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

# dataset_filepaths, dataset_brat_folders = utils.get_valid_dataset_filepaths(parameters)
# dataset = ds.DatasetP(verbose=False, debug=False)
token_to_vector = utils_nlp.load_pretrained_token_embeddings('../../../ML_EntityData/embedding/en/glove.6B.100d.txt')

In [3]:
vocab = Dataset(token_to_vector,100)

In [29]:
tokens,labels = dataset.parse_conll('../../../ML_EntityData/data/en/train.txt')

In [30]:
vocab.build_vocabulary(tokens)

In [31]:
vocab.build_labels(labels)

In [32]:
len(vocab.tokens)

23436

In [33]:
token_indices, character_indices_padded, token_lengths, pattern ,label_indices , label_vector= vocab.transform(tokens,labels)

In [34]:
model = ner_model.BLSTM_CRF( dataset = vocab, token_embedding_dimension = 100 , character_lstm_hidden_state_dimension = 25,
                 token_lstm_hidden_state_dimension = 50 , character_embedding_dimension = 25)

embedded_characters: Tensor("character_embedding/embedded_characters:0", shape=(?, ?, 25), dtype=float32)
embedded_tokens: Tensor("token_embedding/embedding_lookup:0", shape=(?, 100), dtype=float32)
token_lstm_input: Tensor("concatenate_token_and_character_vectors/token_lstm_input:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop: Tensor("dropout/token_lstm_input_drop/mul:0", shape=(?, 163), dtype=float32)
token_lstm_input_drop_expanded: Tensor("dropout/token_lstm_input_drop_expanded:0", shape=(1, ?, 163), dtype=float32)
unary_scores_expanded: Tensor("crf/unary_scores_expanded:0", shape=(1, ?, 12), dtype=float32)
input_label_indices_flat_batch: Tensor("crf/input_label_indices_flat_batch:0", shape=(1, ?), dtype=int32)
sequence_lengths: Tensor("crf/sequence_lengths:0", shape=(1,), dtype=int32)


C:\Users\linhpn.VISC\AppData\Local\Continuum\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [35]:
sess = tf.Session()

In [36]:
sess.run(tf.global_variables_initializer())

In [37]:
sess.run(model.token_embedding_weights.assign(vocab.get_embedding(token_to_vector,100)))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [38]:
model.train_step(sess,token_indices, character_indices_padded, token_lengths, pattern ,label_indices , label_vector ,0.5 )

accuracy :  1 loss :  20.331879
accuracy :  1 loss :  11.125923
accuracy :  1 loss :  19.11522
accuracy :  1 loss :  7.1867523
accuracy :  1 loss :  0.8824196
accuracy :  1 loss :  4.8396873
accuracy :  1 loss :  6.703743
accuracy :  1 loss :  1.5310974
accuracy :  1 loss :  1.9914784
accuracy :  1 loss :  0.012184143
accuracy :  1 loss :  0.478405
accuracy :  1 loss :  7.863373
accuracy :  1 loss :  0.107922554
accuracy :  1 loss :  1.3921509
accuracy :  1 loss :  2.2625046


array([[-8.99765968e-01, -3.37564826e-01, -5.49509168e-01,
        -3.61137897e-01, -3.86134833e-01, -2.47585803e-01,
         3.25152206e+00, -3.11443418e-01, -1.57752708e-01,
         4.29865599e-01, -4.93717074e-01, -4.89027858e-01],
       [ 1.70925748e+00,  7.96052754e-01,  1.58780837e+00,
         1.89074624e+00, -2.01285172e+00,  1.83266199e+00,
        -2.17760420e+00, -1.64479470e+00, -1.23317170e+00,
        -3.76378857e-02, -3.52226973e-01,  5.49042106e-01],
       [ 5.01621515e-02, -2.04380299e-03,  1.36857167e-01,
        -3.78809243e-01, -3.34768891e-02, -2.50578284e-01,
        -3.93024534e-01,  2.57517314e+00, -9.19675746e-04,
        -3.99849117e-01, -1.75878048e-01, -1.29035739e-02],
       [-3.75232369e-01, -4.44905251e-01, -3.26523781e-01,
        -9.34358537e-01,  4.01447916e+00, -8.35896671e-01,
        -7.66457021e-01, -5.00435472e-01,  4.92477007e-02,
         3.32046062e-01, -4.13006306e-01, -4.34453338e-01],
       [-1.32810757e-01,  6.60662830e-01,  5.7881992

In [24]:
model.saver.save(sess, '../../model/model.ckpt')

'../../model/model.ckpt'

In [15]:
vocab.labels

['B-ORG', 'O', 'B-MISC', 'B-PER', 'I-PER', 'B-LOC', 'I-ORG', 'I-MISC', 'I-LOC']

In [7]:
len(vocab.get_embedding(token_to_vector,100))

18416

In [13]:
def predict(text):
    text = text+'.'
    sentences = vocab.tokenizer(text)
    token_indices_test, character_indices_padded_test, token_lengths_test, pattern_test = vocab.transform(sentences)
    pre = model.predict(sess,token_indices_test, character_indices_padded_test, token_lengths_test, pattern_test )
    
    prediction_output =[ [vocab.labels[i] for i in sentence ] for sentence in pre]
       
    tokens=[]
    entitys=[]
    for i,sentence in enumerate(prediction_output):
        token = ''
        previous_label= 'O'
        sentence = utils_nlp.bioes_to_bio(sentence)
        for j,label in enumerate(sentence):
            if label!= 'O':
                label = label.split('-')
                prefix = label[0]
                if prefix == 'B' or previous_label != label[1]:
                    if previous_label != 'O':
                        tokens.append(token)
                        entitys.append(previous_label)
                        token = ''
                    previous_label = label[1]
                    token = sentences[i][j]  #self.dataset.index_to_token[self.dataset.token_indices[dataset_type][i][j]]
                else:
                    token = token + ' ' + sentences[i][j] # self.dataset.index_to_token[self.dataset.token_indices[dataset_type][i][j]]

            else:
                if previous_label != 'O':
                    tokens.append(token)
                    entitys.append(previous_label)
                    token=''
                previous_label = 'O'    
    return list(zip(tokens,entitys))

In [23]:
predict('my name is Ngoc Linh, I am working at Noi')

[('Ngoc Linh', 'PER'), ('Noi', 'ORG')]

([[1051, 3228, 446, 8337, 8338, 11]],
 [[[40, 43, 0, 0],
   [31, 34, 40, 28],
   [33, 29, 0, 0],
   [16, 46, 37, 44],
   [8, 33, 31, 39],
   [19, 0, 0, 0]]],
 [[2, 4, 2, 4, 4, 1]],
 [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
   [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]])

In [12]:
import sys
import pickle

In [8]:
pickle.dump(vocab.get_embedding(token_to_vector,100),open('../../vocab.pickle','wb'))

In [11]:
sys.getsizeof(token_to_vector)

20971616